Course Human-Centered Data Science ([HCDS](https://www.mi.fu-berlin.de/en/inf/groups/hcc/teaching/winter_term_2020_21/course_human_centered_data_science.html)) - Winter Term 2020/21 - [HCC](https://www.mi.fu-berlin.de/en/inf/groups/hcc/index.html) | [Freie Universität Berlin](https://www.fu-berlin.de/)

***

# A4 - Transparency
Please use the follwing structure as a starting point. Extend and change the notebook according to your needs. This structure should help you to guide you through your analysis. This notebook is the foundation for condensing your results and writing your reflection in the end. So please read what we expect from you regarding the reflection first to structure your analysis accordingly.

In [2]:
import pandas as pd
import requests
import json

In [92]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

## [1] General understanding
> What is the model about and who is using it?

* What is your model about?
* Why is this model useful?
* Who is using this model?
* What are stakeholder or users of ORES?
* Why is this model useful to wikipedia?
* What applications/projects/... within wikipedia are using this model?

## [2] API
> What does the ORES API (v3) tell you about a specific model? What functions does the API offer?

Use the API to investigate your model: https://ores.wikimedia.org/v3/#/. What do the follwing API calls do and what do they tell you about your model?

* `https://ores.wikimedia.org/v3/scores/`
* `https://ores.wikimedia.org/v3/scores/?model_info`
* `https://ores.wikimedia.org/v3/scores/enwiki`
* `https://ores.wikimedia.org/v3/scores/enwiki?models=YOURMODELNAME&model_info`
* `https://ores.wikimedia.org/v3/scores/enwiki?models=YOURMODELNAME&revids=SOMEIDHERE`
* `https://ores.wikimedia.org/v3/scores/enwiki/REVID/YOURMODELNAME?model_info`
* `https://ores.wikimedia.org/v3/scores/enwiki/REVID/YOURMODELNAME?features=true`

> The API call `https://ores.wikimedia.org/v3/scores/` does provide information about the projects and the available models and versions for each project. 
_______________________________________________________

In [3]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/')
v3_scores = api_call.json()

In [4]:
accuracy_dict = {'project': [],'model':[],'version':[]}
v3_scores_df = pd.DataFrame(accuracy_dict, columns = ['model','project', 'version' ])

In [5]:
for project in v3_scores.keys():
    for model in v3_scores[project]['models'].keys():
        version = v3_scores[project]['models'][model]['version']
        v3_scores_df = v3_scores_df.append({ 'model':model, 'project': project, 'version':version}, ignore_index=True)

In [6]:
print('ORES API (v3) provides ',len(pd.DataFrame(pd.unique(v3_scores_df['model']))),' different models')
pd.DataFrame(pd.unique(v3_scores_df['model']))

ORES API (v3) provides  10  different models


,0
0,articletopic
1,damaging
2,goodfaith
3,reverted
4,articlequality
5,draftquality
6,drafttopic
7,wp10
8,pagelevel
9,itemquality


In [7]:
print('The model reverted is available in',len(v3_scores_df[v3_scores_df['model'] == 'reverted']),' projects.')
v3_scores_df[v3_scores_df['model'] == 'reverted']

The model reverted is available in 10  projects.


,model,project,version
3,reverted,bnwiki,0.5.0
13,reverted,elwiki,0.5.0
21,reverted,enwiktionary,0.5.0
46,reverted,glwiki,0.5.0
49,reverted,hrwiki,0.5.0
52,reverted,idwiki,0.5.0
53,reverted,iswiki,0.5.0
91,reverted,tawiki,0.5.0
98,reverted,testwiki,0.0.3
108,reverted,viwiki,0.5.0


> The API call `https://ores.wikimedia.org/v3/scores/?model_info` provids information about the models `environment`, `params`, `score_schema`, `statistics`, `type` and `version`.

`environment`: 

________________________________________________

In [103]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/?model_info')
v3_scores_model_info = api_call.json()

In [104]:
pd.DataFrame(v3_scores_model_info['hrwiki']['models'].keys())

,0
0,reverted


In [105]:
v3_scores_model_info['hrwiki']['models']['reverted'].keys()

dict_keys(['environment', 'params', 'score_schema', 'statistics', 'type', 'version'])

In [106]:
def model_overview(project, model):
    env        = pd.DataFrame(v3_scores_model_info[project]['models'][model]['environment'].keys(),columns=['environment'])
    params     = pd.DataFrame(v3_scores_model_info[project]['models'][model]['params'].keys(),columns=['params'] )
    schema     = pd.DataFrame(v3_scores_model_info[project]['models'][model]['score_schema'].keys(), columns=['score_schema'])
    stats     = pd.DataFrame(v3_scores_model_info[project]['models'][model]['statistics'].keys(),columns=['statistics'] )   
    model_overview = params.join([env, stats, schema], how='outer').fillna('')
    return model_overview

In [107]:
model_overview = model_overview('elwiki', 'reverted')

In [108]:
model_overview

,params,environment,statistics,score_schema
0,ccp_alpha,machine,!f1,properties
1,center,platform,!precision,title
2,criterion,processor,!recall,type
3,init,python_branch,accuracy,
4,label_weights,python_build,counts,
5,labels,python_compiler,f1,
6,learning_rate,python_implementation,filter_rate,
7,loss,python_revision,fpr,
8,max_depth,python_version,match_rate,
9,max_features,release,pr_auc,


In [109]:
stats_dict = {'metrics': [],'value':[]}
v3_stats_df = pd.DataFrame(stats_dict, columns = ['metrics','value' ])

for stat in v3_scores_model_info[project]['models'][model]['statistics'].keys():
    for label in v3_scores_model_info[project]['models'][model]['statistics'][stat].keys():
        v3_stats_df = v3_stats_df.append({ 'metrics':stat+' ('+label+')', 
                                          'value': v3_scores_model_info[project]['models'][model]['statistics'][stat][label]}, 
                                         ignore_index=True)

In [93]:
v3_stats_df

,metrics,value
0,!f1 (labels),"{'false': 0.985, 'true': 0.375}"
1,!f1 (macro),0.68
2,!f1 (micro),0.394
3,!precision (labels),"{'false': 0.977, 'true': 0.593}"
4,!precision (macro),0.785
5,!precision (micro),0.605
6,!recall (labels),"{'false': 0.994, 'true': 0.274}"
7,!recall (macro),0.634
8,!recall (micro),0.297
9,accuracy (labels),"{'false': 0.971, 'true': 0.971}"


> The API call `https://ores.wikimedia.org/v3/scores/enwiki` provides information about available models and version iformation for the `enwiki` project. 

________________________________________________

In [16]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/enwiki')
v3_scores_enwiki = api_call.json()

In [17]:
pd.DataFrame.from_dict(v3_scores_enwiki['enwiki']['models'])

,articlequality,articletopic,damaging,draftquality,drafttopic,goodfaith,wp10
version,0.8.2,1.2.0,0.5.1,0.2.1,1.2.0,0.5.1,0.8.2


The API call `https://ores.wikimedia.org/v3/scores/enwiki?models=reverted&model_info` provides the same information as the call `https://ores.wikimedia.org/v3/scores/?model_info` provides. But with this call the information can be requested for the specific project and model where with the other API call information is returned for all projects and models. 

If the asked model is not available for the project then an error is returned that it does not exist for the specified project: 

'code': 'not found',
 'message': "Models ('reverted',) not available for enwiki

________________________________________________

In [117]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki?models=reverted&model_info')
v3_scores_reverted_modelinfo = api_call.json()

In [119]:
v3_scores_reverted_modelinfo['hrwiki']['models']['reverted'].keys()

dict_keys(['environment', 'params', 'score_schema', 'statistics', 'type', 'version'])

In [123]:
stats_dict = {'metrics': [],'value':[]}
v3_stats_df = pd.DataFrame(stats_dict, columns = ['metrics','value' ])

for stat in v3_scores_reverted_modelinfo['hrwiki']['models']['reverted']['statistics'].keys():
    for label in v3_scores_reverted_modelinfo['hrwiki']['models']['reverted']['statistics'][stat].keys():
        v3_stats_df = v3_stats_df.append({ 'metrics':stat+' ('+label+')', 
                                          'value': v3_scores_reverted_modelinfo['hrwiki']['models']['reverted']['statistics'][stat][label]}, 
                                         ignore_index=True)

In [124]:
v3_stats_df

,metrics,value
0,!f1 (labels),"{'false': 0.494, 'true': 0.919}"
1,!f1 (macro),0.707
2,!f1 (micro),0.527
3,!precision (labels),"{'false': 0.347, 'true': 0.986}"
4,!precision (macro),0.666
5,!precision (micro),0.398
6,!recall (labels),"{'false': 0.855, 'true': 0.862}"
7,!recall (macro),0.858
8,!recall (micro),0.855
9,accuracy (labels),"{'false': 0.861, 'true': 0.861}"


> The API call `https://ores.wikimedia.org/v3/scores/enwiki?models=reverted&revids=235107991` returns a prediction for a specific wikipedia article revision if this problbly needs to be reverted or not. 

________________________________________________

In [22]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki?models=reverted&revids=5618117')
v3_scores_reverted_revid = api_call.json()

In [24]:
v3_scores_reverted_revid['hrwiki']

{'models': {'reverted': {'version': '0.5.0'}},
 'scores': {'5618117': {'reverted': {'score': {'prediction': False,
     'probability': {'false': 0.8265831588525849,
      'true': 0.17341684114741504}}}}}}

> The API call `https://ores.wikimedia.org/v3/scores/enwiki/REVID/YOURMODELNAME?model_info` returns information about the model and the score for the spcified article revision. I think is is especialy usefull when checkin some performance metrices like 
the precision or recall is important to together with the score. 

________________________________________________

In [48]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?model_info')
v3_scores_reverted_modelinfo_revid = api_call.json()

In [49]:
v3_scores_reverted_modelinfo_revid['hrwiki']['scores']['5618117']['reverted']

{'score': {'prediction': False,
  'probability': {'false': 0.8265831588525849, 'true': 0.17341684114741504}}}

In [51]:
v3_scores_reverted_modelinfo_revid['hrwiki']['scores']

{'5618117': {'reverted': {'score': {'prediction': False,
    'probability': {'false': 0.8265831588525849,
     'true': 0.17341684114741504}}}}}

> The API call `https://ores.wikimedia.org/v3/scores/elwiki/807457197/reverted?features=true` returns information about the model and the score for the spcified article revision. I think is is especialy usefull when checkin some performance metrices like 
the precision or recall is important to together with the score. 

In [25]:
api_call = requests.get('https://ores.wikimedia.org/v3/scores/hrwiki/5618117/reverted?features=true')
v3_scores_reverted_features_revid = api_call.json()

In [34]:
v3_scores_reverted_features_revid['hrwiki']['scores']['5618117']['reverted'].keys()

dict_keys(['features', 'score'])

In [43]:
feature_dict = {'feature': [],'value':[]}
v3_features_df = pd.DataFrame(feature_dict, columns = ['feature','value' ])

for feature in v3_scores_reverted_features_revid['hrwiki']['scores']['5618117']['reverted']['features'].keys():
    v3_features_df = v3_features_df.append({ 'feature': feature, 'value':v3_scores_reverted_features_revid['hrwiki']['scores']['5618117']['reverted']['features'][feature] }, ignore_index=True)

In [91]:
v3_features_df[:50].style.set_properties(**{'text-align': 'left'})

,feature,value
0,feature.croatian.badwords.revision.diff.match_delta_decrease,0.000000
1,feature.croatian.badwords.revision.diff.match_delta_increase,0.000000
2,feature.croatian.badwords.revision.diff.match_delta_sum,0.000000
3,feature.croatian.badwords.revision.diff.match_prop_delta_decrease,0.000000
4,feature.croatian.badwords.revision.diff.match_prop_delta_increase,0.000000
5,feature.croatian.badwords.revision.diff.match_prop_delta_sum,0.000000
6,feature.croatian.informals.revision.diff.match_delta_decrease,0.000000
7,feature.croatian.informals.revision.diff.match_delta_increase,0.000000
8,feature.croatian.informals.revision.diff.match_delta_sum,0.000000
9,feature.croatian.informals.revision.diff.match_prop_delta_decrease,0.000000


### Feature Injection
Please check out the _feature injection_ feature of ORES: https://www.mediawiki.org/wiki/ORES/Feature_injection

**Example:**

     # Here you can get the perdiction for a revision, if the user would habe been anonymous:
     https://ores.wikimedia.org/v3/scores/enwiki/991397091/damaging?features&feature.revision.user.is_anon=true

## [3] ML algorithm and training/test data
> Which machine learning model is underlying and what data is used to build the model?

* Check out `model_info` in detail.
* What does it tell you about the model performance?
* You can visualise and explain your results regarding model performance.
* What data was used to train and test the model?
* What machine learning algorithm is your model using? Please explain briefly.

## [4] Features
> Which features are used and which have the greatest influence on the prediction?

* What features is your model using?
* What do they mean?
* Which is the most important features?
* `https://ores.wikimedia.org/v3/scores/enwiki/991379667/articlequality?features=true`
* Are all models (in all languages of wikipedia), are they using the same features?

## Sample code

In [ ]:


# Customize these with your own information
headers = {
    'User-Agent': 'https://github.com/YOUR-USER-NAME',
    'From': 'YOUR-EMAIL@fu-berlin.de'
}

def get_ores_data(rev_id, headers):
    
    # Define the endpoint: This is an example!
    endpoint = 'https://ores.wikimedia.org/v3/scores/{project}/?models={model}&revids={revids}'

    params = {'project' : 'enwiki',
              'model'   : 'YOUMODELNAME',
              'revids'  : rev_id
              }

    api_call = requests.get(endpoint.format(**params))
    response = api_call.json()
    data = json.dumps(response)

    return data

***

#### Credits

We release the notebooks under the [Creative Commons Attribution license (CC BY 4.0)](https://creativecommons.org/licenses/by/4.0/).